### Lesson 4: Vectorstores and Embeddings

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
loaders = [
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture03.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture03.pdf"),
]

docs: list = []

for loader in loaders:
    docs.extend(loader.load())

In [4]:
len(docs)

76

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
)

splits = text_splitter.split_documents(docs)

In [6]:
len(splits)

208

### Embeddings

In [7]:
from langchain.embeddings.cohere import CohereEmbeddings

In [8]:
embedding = CohereEmbeddings()

In [9]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [10]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [11]:
import numpy as np

In [12]:
np.dot(embedding1, embedding2)

7524.555609095167

In [13]:
np.dot(embedding1, embedding3)

2642.9197042219853

In [14]:
np.dot(embedding2, embedding3)

2150.6375048579607

### Vectorstores

In [15]:
from langchain.vectorstores.chroma import Chroma

In [16]:
persist_directory = "./.chroma/"

In [17]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory,
)

In [18]:
vectordb._collection.count()

208

### Similarity Search

In [19]:
question = "is there an email i can ask for help"

In [20]:
docs = vectordb.similarity_search(query=question, k=3)

In [21]:
len(docs)

3

In [22]:
print(docs[0].page_content)

cs229-qa@cs.stanford.edu. This goes to an acc ount that's read by all the TAs and me. So 
rather than sending us email individually, if you send email to this account, it will 
actually let us get back to you maximally quickly with answers to your questions.  
If you're asking questions about homework probl ems, please say in the subject line which 
assignment and which question the email refers to, since that will also help us to route 
your question to the appropriate TA or to me  appropriately and get the response back to 
you quickly.  
Let's see. Skipping ahead — let's see — for homework, one midterm, one open and term 
project. Notice on the honor code. So one thi ng that I think will help you to succeed and 
do well in this class and even help you to enjoy this cla ss more is if you form a study 
group.  
So start looking around where you' re sitting now or at the end of class today, mingle a 
little bit and get to know your classmates. I strongly encourage you to form study gro

In [23]:
print(docs[1].page_content)

cs229-qa@cs.stanford.edu. This goes to an acc ount that's read by all the TAs and me. So 
rather than sending us email individually, if you send email to this account, it will 
actually let us get back to you maximally quickly with answers to your questions.  
If you're asking questions about homework probl ems, please say in the subject line which 
assignment and which question the email refers to, since that will also help us to route 
your question to the appropriate TA or to me  appropriately and get the response back to 
you quickly.  
Let's see. Skipping ahead — let's see — for homework, one midterm, one open and term 
project. Notice on the honor code. So one thi ng that I think will help you to succeed and 
do well in this class and even help you to enjoy this cla ss more is if you form a study 
group.  
So start looking around where you' re sitting now or at the end of class today, mingle a 
little bit and get to know your classmates. I strongly encourage you to form study gro

In [24]:
print(docs[2].page_content)

more fun for you, and you'd probably have a be tter learning experience if you form a 
study group of people to work with. So I definitely encourage you to do that.  
And just to say a word on the honor code, whic h is I definitely en courage you to form a 
study group and work together, discuss homew ork problems together. But if you discuss


In [25]:
vectordb.persist()

### Failure modes

This seems great, and basic similarity search will get you 80% of the way there very easily.

But there are some failure modes that can creep up.

In [26]:
question = "What did they say about Matlab?"

In [27]:
docs = vectordb.similarity_search(query=question, k=5)

In [28]:
docs[0]

Document(page_content='those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people call it a free ve rsion of MATLAB, which it sort  of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t s een MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your  own home computer or something if you \ndon\'t have a MATLAB license, for the purposes of  this class, there\'s also — [inaudible] \nwrite that down [inaudible] MATLAB — there\' s also a software package called Octave \nthat you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it\'s free, and for the purposes of  this class,

In [29]:
docs[1]

Document(page_content='those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people call it a free ve rsion of MATLAB, which it sort  of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t s een MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your  own home computer or something if you \ndon\'t have a MATLAB license, for the purposes of  this class, there\'s also — [inaudible] \nwrite that down [inaudible] MATLAB — there\' s also a software package called Octave \nthat you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it\'s free, and for the purposes of  this class,

In [30]:
question = "What did they say about regression in the third lecture?"

In [31]:
docs = vectordb.similarity_search(query=question, k=5)

In [32]:
for doc in docs:
    print(doc.metadata)

{'page': 0, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 6, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 6, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 14, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}


In [33]:
print(docs[4].page_content)

Student: It’s the lowest it –  
Instructor (Andrew Ng) :No, exactly. Right. So zero to the same, this is not the same, 
right? And the reason is, in logi stic regression this is diffe rent from before, right? The 
definition of this H subscript theta of XI is not the same as the definition I was using in 
the previous lecture. And in pa rticular this is no longer thet a transpose XI. This is not a 
linear function anymore. This is  a logistic function of theta transpose XI. Okay? So even 
though this looks cosmetically similar, even though this is similar on the surface, to the 
Bastrian descent rule I derive d last time for least squares regression this is actually a 
totally different learning algorithm. Okay? And it turns out that there’s actually no 
coincidence that you ended up with the same l earning rule. We’ll actually talk a bit more 
about this later when we talk about generalized linear models. But this is one of the most 
elegant generalized learning models that we’l l see